# 将上海市旅游数据月分为日

## 1.数据准备
原则：将之前预测的2019全年数据分配作为上海市旅游入境疫情前全年数据日分配，2020-2024依照异常数据处理原则每年一分配

In [ ]:
from enum import Enum
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import torch

# 创建数据库引擎
table_hk_imputed = "hk_customs_imputed"
database_url = "sqlite:///../data/data.sqlite"
engine = create_engine(database_url)

# define hyper paras
use_standard_scaler = True
mode = 'predict'
model_name = 'transformer'  # transformer,saits
save_imputed_data = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
saved_model = {
    "saits": 'saits_hk_imputation_20240222235322.pypots',
    "transformer": "transformer_hk_imputation_20240303162108.pypots"
}
